## Lab 1 Part III: Satellite Data
#### 9/7/2022


The following tutorial is the third of three parts of the Python portion of Lab 1.  In this part, we create GOES-16 satellite  images using the Siphon and matplotlib modules.
<br />

### Module Documentation

1. The datetime function from the datetime module: https://docs.python.org/3/library/datetime.html
2. Siphon TDSCatalog: https://unidata.github.io/siphon/latest/examples/Basic_Usage.html
3. Cartopy Feature: https://scitools.org.uk/cartopy/docs/latest/matplotlib/feature_interface.html
4. Matplotlib: https://matplotlib.org/
5. Matplotlib Normalize: https://matplotlib.org/stable/api/_as_gen/matplotlib.colors.Normalize.html


<br /><br />

If you have any questions about the code below, always feel free to reach out to me at mpvossen@uwm.edu. I am always willing to further explain the code. <br /> <br />

1. To start creating our satellite plots, we first need to import the necessary modules:
<br />


In [ ]:
#this time we want to import everything from the metpy module
import metpy

#from the plotting portion of metpy import the add_timestamp function
from metpy.plots import add_timestamp

#import the crs part of the cartopy module.  This is for handling the data's native and desired map projections.
import cartopy.crs as ccrs

#import the feature part of the cartopy module.  This is for adding map elements such as state lines.
import cartopy.feature as cfeature

#import the pyplot part of the matplotlib module.  This is main Python plotting module.
import matplotlib.pyplot as plt

#import the datetime feature of the datetime module.  This is for working with dates and times.
from datetime import datetime

#from the siphon module, in the catalog part of siphon, import the TDSCatalog class.  This is for accessing data online form a specific type
#of server that is sometime used in atmospheric science called a THREDDS server.  You can think of THREDDS as a website that has a lot of raw data.
from siphon.catalog import TDSCatalog

#from the matplotlib module, in the colors part, import the Normalize function.  This is for adjusting the colors we use to plot the satellite data
from matplotlib.colors import Normalize

#this is for the Jupyter Notebook so that the plot is created within the notebook itself.
%matplotlib inline

<br /><br />
2. Once we have the necessary modules loaded, we can download the needed data.  We are going to access our satellite data from the Unidata Jumbo THREDDS server using one of the modules we have imported.  The Unidata Jumbo THREDDS server has various meteorological datasets ranging from model forecasts (such as the latest GFS model run) to observations (such as GOES-16 data). I will start with the visible image.  <br /><br /> 

In the code section below, I use the TDSCatalog class to access the THREDDS server.  Next, using the response from that server, I grab the latest file.  <br /><br /> If you are interested seeing what data you can access at the Unidata Jumbo THREDDS server, go to this link:  https://thredds-jumbo.unidata.ucar.edu/thredds/catalog.html.
<br />

In [ ]:
#this is a string of the url to the THREDDS server that we are going to be using.  This link is specific to the GOES 16 CONUS Visible satellite image
url = "https://thredds-jumbo.unidata.ucar.edu/thredds/catalog/satellite/goes/east/grb/ABI/CONUS/Channel02/current/catalog.xml"

#here we go to the url and get a list of files that are available to us.  I then save the list to the variable sat_catalog
sat_catalog = TDSCatalog(url)

#we want the latest file so we take our list of files (sat_catalog), get the online locations of the data (.datasets) for specifically the latest file ([0]), and save it to the variable sat_file_name.  
#The list generated before is ordered based on how old the data is, with the newest file being index 0 and the oldest being the last index. 
sat_file_name = sat_catalog.datasets[0]

#now that we have the location of the file, let's open it.  We take the location from before (sat_file_name) and use the remote_access feature This is a special way of opening a file over the internet.  
#Here Here we never download the file, but rather the information goes directly to the memory (RAM) which keeps your file space clean and makes your code faster.
#We also use a package called xarray to open the data from the memory
sat_data = sat_file_name.remote_access(use_xarray=True)

#display the sat_data dataset.  Notice that an xarray dataset is displayed below.  I'll discuss more about it in the cell below.
sat_data

<br /><br />
3.  Now the satellite dataset is open.  Above, you should see a list of coordinates and data variables.  The satellite data are structed that there is coordinate variables that tell you things such as x pixel position, y pixel position, and time.  There are also data variables such as brightness, which is measured by the GOES-16 satellite's visible imager, at the location that corresponds with our x pixel position and y pixel position.  Finally, there are attribute data that describe the data. <br /><br /> 

In it's current format, the satellite data are not in a form that we can plot on a map, so we need to do some data wrangling to get it there.  Luckily, the MetPy module provides code to do this for us.  Here, I use MetPy's parsing code to get the "Rad" variable.  In this file the "Rad" (short for radiance) variable contains our visible light brightness value.<br />

In [ ]:
#for the satellite data (sat_data), using metpy (.metpy) parse out the satellite file (.parse_cf()) specifically for the radiance variable ('Rad')
data = sat_data.metpy.parse_cf('Rad')

<br /><br />
4.  Next, we need to query the data to identify the specific map projection it uses to represent the data. We can get this directly from the data's attributes.  Note that map projections will be discussed in more detail during lecture the week of Sept. 12th. <br />

In [ ]:
#from the parsed data file (data) use metpy (.metpy) to parse the data and get the cartopy_crs attribute variable.  Cartopy_crs is the variable that contains the 
#map projection that the data is designed for.  Outside of satellite data the map projection variable name may be a different name.
proj = data.metpy.cartopy_crs

<br /><br />
5. Now, we extract the x (relating to east-west position) and y (relating to north-south position) coordinate variables from our data.  We will use these later to set our satellite image's spatial extent. <br />

In [ ]:
#from the parsed data file (data) pull out the variable x ([x]).  This is the x location of each point contained in the radiance variable
x = data['x']
#from the parsed data file (data) pull out the variable y ([t]).  This is the y location of each point contained in the radiance variable
y = data['y']

<br /> <br />
6. Here, I set up the colors to be used when plotting the data.  First, I set the range of brightness values that we want to plot.  I choose -25 as a minimum value and allow the maximum radiance value to be the maximum value contained in the data.  The lowest-possible radiance value is 0, but I choose -25 to shift the color table's specified colors to make the land more visible.  <br /><br />  

I also set the color map to the color map gist_yarg_r.  This color map is the reversed version of the gist_yarg color map defined by the matplotlib module.  I reverse the colormap because the default version of this color map makes low brightness (corresponding to dark conditions) white and high brightness (corresponding to bright clouds) black, the opposite of how we typically interpret visible satellite imagery.<br /><br />

To see what this colormap looks like or to see the other color maps available, go to: https://matplotlib.org/stable/tutorials/colors/colormaps.html.<br />

In [ ]:
#set the range that the color table should cover.  Normalize(vmin = minimum color value, vmax= maximum color value)
color_range = Normalize(vmin=-25, vmax=data.max())

#set the matplotlib color map
color_map = 'gist_yarg_r'

<br /><br />
7. We are now ready to plot our satellite data.  Since all plotting code must be in one cell in Jupyter Notebooks, I explain the code in more detail in the comments below.  <br />

In [ ]:
#here I create the figure.  Think of it as the sheet of paper that you are going to draw the map on. 
#I set the figure size to be 1500px X 1500 px and I set the resolution (dpi) to 300 dots per square inch.  A figure dpi of 300 is standard for publications.
#plt.figure(figsize = (the size of the figure), dpi = the figure resolution)
fig = plt.figure(figsize=(15, 15), dpi=300)

#We now need to set our map's projection.  Here we pass in the previously parsed data.
#this function allows us to have multiple maps with plots in our figure but here we only need 1.
#fig.add_subplot(# plot rows, # plot columns, the current plot id, projection = the map projection for the plot)
#the function outputs a blank map plot which I save to ax.
ax = fig.add_subplot(1, 1, 1, projection=proj)

#We now need geographic reference data to help us know where exactly the clouds are.  Here I add the coastlines, states, and country borders.
#I also specify that I want the various borders to have a data resolution of 50 min (where 1 min = 1/60th of a degree of latitude or longitude),
#with a linewidth of 1, and a line (or edge) color of grey.
ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=1, edgecolor = 'grey')
#for the states to make them stand out, I used the dotted linestyle
ax.add_feature(cfeature.STATES.with_scale('50m'), linestyle=':',linewidth = 1, edgecolor='grey')
ax.add_feature(cfeature.BORDERS.with_scale('50m'), linewidth=1, edgecolor='grey')

#this is the actual plot function for the satellite data. (.imshow)
#we pass in the full satellite dataset.  It can automatically identify the variable it needs to plot within the dataset.  
#We also set to extent to be the full extent.  If this wasn't here, there would be extra area in the map that we don't have an image for
#We also pass what colormap we want and what range to which the color map should be set.  We set this up earlier.
im = ax.imshow(data, extent=(x.min(), x.max(), y.min(), y.max()), origin='upper', cmap=color_map, norm=color_range)

#This next line gets the time that the satellite image is valid for from the data file.  This is set up to parse the date in the format it is in, in the datafile.
#datetime.strptime is used for parsing the datetime.  The datetime I pass in comes from the satellite data (data), and I tell Python to take out the time variable (.t) 
#and then get the value contained in that variable (.values).  I then use the string function to make sure the time variable is a string so the datetime parser can work and I remove the last three characters since
#satellite data has a very accurate time stamp that has the seconds to 9 decimal places.  Python can only handle 6 decimal places and thus we just get rid of the last three decimal places.
start_time = datetime.strptime(str(data.t.values)[:-3], 
                               '%Y-%m-%dT%H:%M:%S.%f') #this tells the function how to decode the datetime.  Each % then the letter after, tells the function what 
                                                       #number of the date is located in that part of the string we are decoding the datetime for.  See the datetime documentation
                                                       #for help in decoding each % and letter combination.

#I now insert the time the satellite image is valid for into the image
#add_timestamp(the plot variable, the datetime that the image is for, extra information for before the timestamp, should the text coloring be high contract, font size, vertical position of the text in the plot)

add_timestamp(ax, time=start_time, pretext='GOES-16 Visible Image ',
              high_contrast=True, fontsize=16, y=0.01)

#finally I tell Python to display the created image.
plt.show()

<br /><br />
8. Now, let's create a longwave infrared satellite image.  Below I download the latest satellite image using the same workflow as for the visible satellite image.  Note that the color map range has changed to 10 and 150 Wm<sup>-2</sup>.  Raw satellite data like we are working with is always measured in radiances and the imagery we normally see in websites for infrared imagery is a temperature that was derived from the measured radiances.

In [ ]:
#this is the url to the THREDDS server that we are going to be using.  This link is specific to the GOES 16 CONUS longwave infrared satellite image.
url = "https://thredds-jumbo.unidata.ucar.edu/thredds/catalog/satellite/goes/east/grb/ABI/CONUS/Channel13/current/catalog.xml"
#here we go to the url and see what satellite images are available to us
sat_catalog = TDSCatalog(url)

#we want the latest file so we select the first file (index 0)
sat_file_name = sat_catalog.datasets[0]

#now that we have the name of the file, let's open it.  This is a special way of opening a file over the internet.  Here we never download the file, but rather
#the information goes directly to the memory.
#We also use a package called xarray to open the data from the memory
sat_data = sat_file_name.remote_access(use_xarray=True)

#parse the satellite data
data = sat_data.metpy.parse_cf('Rad')

#get the projection
proj = data.metpy.cartopy_crs

#pull out the x and y coordinates
x = data['x']
y = data['y']

#set color range
color_range = Normalize(vmin=10, vmax=150)



<br /><br />
9. In the section below, using the reference I gave in =section 6 of this tutorial, choose a white to black or black to white colormap other than gist_yarg for your image.  Since are working with longwave infrared imagery, be sure to choose a color map with low values being white and high values being black.  <br /><br />Note: You can flip any color map by adding _r to the end of its name. <br />

In [ ]:
color_map = 'gist_yarg'

<br /><br />
10. Now we're ready to plot the longwave infrared satellite imagery.  Before doing so, however, change the time stamp text in the code block below so it says longwave infrared rather than visible.<br />

In [ ]:
#here I create the figure.  Think of it as the sheet of paper that you are going to draw the map on. 
#I set the figure size to be 1500px X 1500 px and I set the resolution (dpi) to 300 dots per square inch.  A figure dpi of 300 is standard for publications.
#plt.figure(figsize = (the size of the figure), dpi = the figure resolution)
fig = plt.figure(figsize=(15, 15), dpi=300)

#We now need to set what projection we are plotting our map on.  Here we pass on the data we parsed out before.
#this function allows us to have multiple maps with plots in our figure but here we only need 1.
#fig.add_subplot(# plot rows, # plot columns, the current plot id, projection = the map projection for the plot)
#the function outputs a blank map plot which I save to ax.
ax = fig.add_subplot(1, 1, 1, projection=proj)

#We now need geographic reference data to help us know where exactly the clouds are.  Here I add the coastlines, states, and country borders.
#I also specify that I want the various borders to have a data resolution of 50 min (where 1 min = 1/60th of a degree of latitude or longitude),
#with a linewidth of 1, and a line (or edge) color of grey.
ax.add_feature(cfeature.COASTLINE.with_scale('50m'), linewidth=1, edgecolor = 'grey')
#for the states to make them stand out, I used the dotted linestyle
ax.add_feature(cfeature.STATES.with_scale('50m'), linestyle=':',linewidth = 1, edgecolor='grey')
ax.add_feature(cfeature.BORDERS.with_scale('50m'), linewidth=1, edgecolor='grey')

#this is the actual plot function for the satellite data. (.imshow)
#we pass in the full satellite dataset.  It can automatically identify the variable it needs to plot within the dataset.  
#We also set to extent to be the full extent.  If this wasn't here, there would be extra area in the map that we don't have an image for
#We also pass what colormap we want and what range to which the color map should be set.  We set this up earlier.
im = ax.imshow(data, extent=(x.min(), x.max(), y.min(), y.max()), origin='upper', cmap=color_map, norm=color_range)

#This next line gets the time that the satellite image is valid for from the data file.  This is setup to parse the date in the format it is in, in the datafile.
#datetime.strptime is used for parsing the datetime.  The datetime I pass in comes from the satellite data (data), and I tell Python to take out the time variable (.t) 
#and then get the value contained in that variable (.values).  I then use the string function to make sure the time variable is a string so the datetime parser can work and I remove the last three characters since
#satellite data has a very actuate time stamp that has the seconds to 9 decimal places.  Python can only handle 6 decimal places and thus we just get rid of the last three decimal places.
start_time = datetime.strptime(str(data.t.values)[:-3], 
                               '%Y-%m-%dT%H:%M:%S.%f') #this tells the function how to decode the datetime.  Each % then the letter after, tells the function what 
                                                       #number of the date is located in that part of the string we are decoding the datetime for.  See Lab 1 appendix
                                                       #for what each % and letter combination stands for

#I now insert the time the satellite image is valid for into the image
#add_timestamp(the plot variable, the datetime that the image is for, extra information for before the timestamp, should the text coloring be high contract, font size, vertical position of the text in the plot)

add_timestamp(ax, time=start_time, pretext='GOES-16 Visible Image ',
              high_contrast=True, fontsize=16, y=0.01)

#finally I tell Python to display the created image.
plt.show()

<br /><br />

You have now completed the third and part of the Python portion of Lab 1.  Be sure to submit the fully rendered Jupyter Notebook on GitHub when you are finished.
